In [2]:
import os
from tqdm import tqdm
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model

In [3]:
#load the model
model = EfficientNetB3()

#model restructure
#the purpose of using -2 in layer function is because we're not using last prediction layer i.e layer -1
#and the layer -2 is the layer before the prediction layer which we'll be using for output
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

#summary of model
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 300, 300,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 300, 300,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 300, 300,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │      1,080 │ stem_conv_pad[0]… │
│                     │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        160 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        360 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        160 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 40)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 40)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 40)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 10)  │        410 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 40)  │        440 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 40)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        960 │ block1a_se_excit

 Total params: 10,783,535 (41.14 MB)

 Trainable params: 10,696,232 (40.80 MB)

 Non-trainable params: 87,303 (341.03 KB)

In [4]:
path = "/home/saurav/Documents/ai-powered-ecommerce/images/train"

In [5]:
features = {}
for img in tqdm(os.listdir(path)):
    image_path = os.path.join(path,img)
    #print(image_path)

    #loading the image
    img = load_img(image_path, target_size=(300,300))

    #converting image to numpy array
    img = img_to_array(img)

    #print(img.shape)
    #reshape the image for the model
    #here reshape is used to add the batch size dimension
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])

    #image preprocessing
    img = preprocess_input(img)

    #feature extraction
    feature = model.predict(img, verbose=0)
    feature = feature.flatten()

    #feature extraction
    features[image_path] = feature

100%|████████████████████████████████████████████████| 91166/91166 [5:02:52<00:00,  5.02it/s]


In [11]:
feature_information = features

In [12]:
feature_information

{'/home/saurav/Documents/ai-powered-ecommerce/images/train/27247.jpeg': array([ 0.03424488,  0.12498824,  0.02502169, ..., -0.05879436,
         0.01568343, -0.1196274 ], dtype=float32),
 '/home/saurav/Documents/ai-powered-ecommerce/images/train/28592.jpeg': array([ 0.33409232,  0.22364983, -0.20059559, ..., -0.08358056,
         0.07339779, -0.17781706], dtype=float32),
 '/home/saurav/Documents/ai-powered-ecommerce/images/train/34384.jpeg': array([ 0.4867844 , -0.01743349,  0.03592414, ..., -0.1606089 ,
        -0.14527327,  0.37913468], dtype=float32),
 '/home/saurav/Documents/ai-powered-ecommerce/images/train/5831.jpeg': array([ 0.04466137, -0.12839247, -0.16591147, ..., -0.02500007,
         0.0325628 , -0.02028537], dtype=float32),
 '/home/saurav/Documents/ai-powered-ecommerce/images/train/54800.jpg': array([ 0.0269615 , -0.14921398, -0.13427092, ..., -0.19779302,
        -0.11751454,  0.5416225 ], dtype=float32),
 '/home/saurav/Documents/ai-powered-ecommerce/images/train/12611.jp